In [ ]:
# Trabalho Referência
# https://jose-llorens-ripolles.medium.com/stockpile-volume-with-open3d-fa9d32099b6f

In [1]:
import open3d as o3d
import numpy as np
from scipy.spatial import Delaunay, ConvexHull
import random
import math
from functools import reduce

In [ ]:
pcd = o3d.io.read_point_cloud("..\pilha.ply")
axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd, axes])

# scale_factor = 6.1579  
# pcd.points = o3d.utility.Vector3dVector(np.asarray(pcd.points) * scale_factor)

# axes = o3d.geometry.TriangleMesh.create_coordinate_frame()
# o3d.visualization.draw_geometries([pcd, axes])

In [64]:
# Segmentação de plano
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=50000)
[a, b, c, d] = plane_model
plane_pcd = pcd.select_by_index(inliers)
plane_pcd.paint_uniform_color([1.0, 0, 0])
stockpile_pcd = pcd.select_by_index(inliers, invert=True)
stockpile_pcd.paint_uniform_color([0, 0, 1.0])
o3d.visualization.draw_geometries([plane_pcd, stockpile_pcd, axes])

In [65]:
# Transformações geométricas
plane_pcd = plane_pcd.translate((0, 0, d/c))
stockpile_pcd = stockpile_pcd.translate((0, 0, d/c))
cos_theta = c / math.sqrt(a**2 + b**2 + c**2)
sin_theta = math.sqrt((a**2 + b**2)/(a**2 + b**2 + c**2))
u_1 = b / math.sqrt(a**2 + b**2)
u_2 = -a / math.sqrt(a**2 + b**2)
rotation_matrix = np.array([[cos_theta + u_1**2 * (1-cos_theta), u_1*u_2*(1-cos_theta), u_2*sin_theta],
                            [u_1*u_2*(1-cos_theta), cos_theta + u_2**2*(1-cos_theta), -u_1*sin_theta],
                            [-u_2*sin_theta, u_1*sin_theta, cos_theta]])
plane_pcd.rotate(rotation_matrix)
stockpile_pcd.rotate(rotation_matrix)
o3d.visualization.draw_geometries([plane_pcd, stockpile_pcd, axes])

In [66]:
o3d.visualization.draw_geometries([stockpile_pcd])

In [67]:
# Remover outliers
cl, ind = stockpile_pcd.remove_statistical_outlier(nb_neighbors=30,
                                                    std_ratio=2.0)
stockpile_pcd = stockpile_pcd.select_by_index(ind)
o3d.visualization.draw_geometries([stockpile_pcd])

In [68]:
# Downsampling e triangulação
downpdc = stockpile_pcd.voxel_down_sample(voxel_size=0.01)
xyz = np.asarray(downpdc.points)
xy_catalog = []
for point in xyz:
    xy_catalog.append([point[0], point[1]])
tri = Delaunay(np.array(xy_catalog))
xy_catalog = np.array(xy_catalog)

In [69]:
# Criar superfície
surface = o3d.geometry.TriangleMesh()
surface.vertices = o3d.utility.Vector3dVector(xyz)
surface.triangles = o3d.utility.Vector3iVector(tri.simplices)
o3d.visualization.draw_geometries([surface], mesh_show_wireframe=True)

In [70]:
# Funções para cálculo de volume
def get_triangles_vertices(triangles, vertices):
    triangles_vertices = []
    for triangle in triangles:
        new_triangles_vertices = [vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]]]
        triangles_vertices.append(new_triangles_vertices)
    return np.array(triangles_vertices)

def volume_under_triangle(triangle):
    p1, p2, p3 = triangle
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    x3, y3, z3 = p3
    return abs((z1+z2+z3)*(x1*y2-x2*y1+x2*y3-x3*y2+x3*y1-x1*y3)/6)

In [71]:
# Calcular volume
volume = reduce(lambda a, b: a + volume_under_triangle(b), 
                get_triangles_vertices(surface.triangles, surface.vertices), 0)
print(f"The volume of the stockpile is: {round(volume, 2)} m3")

The volume of the stockpile is: 11.33 m3


In [72]:
densidade_g_cm3 = 2.98 
densidade_kg_m3 = densidade_g_cm3 * 1000

In [73]:
def calcular_massa(volume, densidade):
    return volume * densidade

def formatar_massa(massa_kg):
    return {'kg': massa_kg, 'toneladas': massa_kg / 1000}

In [ ]:
# Calcular massa
massa_kg = calcular_massa(volume, densidade_kg_m3)
massa_formatada = formatar_massa(massa_kg)

print(f"Volume da pilha: {volume:,.2f} m³")
print(f"\nDensidade informada: {densidade_g_cm3} g/cm³")
print(f"Densidade convertida: {densidade_kg_m3:,.2f} kg/m³")
print(f"\nMassa total: {massa_formatada['kg']:,.2f} kg")
print(f"Massa total: {massa_formatada['toneladas']:,.2f} toneladas")